<a href="https://colab.research.google.com/github/joao1barbosa/ml-lab/blob/main/Predi%C3%A7%C3%A3o_de_gera%C3%A7%C3%A3o_Bruky.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação de bibliotecas


In [1]:
!pip install pandas numpy matplotlib

Importação das bibliotecas


In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

Importação dos dados de geração de energia de Outubro de 2023 a Outubro de 2024

In [4]:
df = pd.read_csv('geracao_energia.csv')
print(df)

       Data  Energia gerada(kWh)
0   2023/10             73527.14
1   2023/11             76311.63
2   2023/12             79446.45
3   2024/01             80503.63
4   2024/02             66664.83
5   2024/03             59461.22
6   2024/04             62226.98
7   2024/05             55832.88
8   2024/06             47758.45
9   2024/07             48878.33
10  2024/08             56884.30
11  2024/09             70679.71
12  2024/10             67896.32


Importação dos dados de clima de Jataí por hora, de Outubro de 2023 a Outubro 2024, e filtra eles para apresentá-los melhor.

In [5]:
df_clima = pd.read_csv('clima_jatai_08-2023_a_08-2024.csv', sep=';', encoding='latin-1')
df_clima = df_clima.iloc[:, :-1]
df_clima.columns = [
    "Data", "Hora UTC", "Precipitação (mm)", "Pressão (mB)",
    "Pressão Máx (mB)", "Pressão Mín (mB)", "Radiação (Kj/m²)",
    "Temp. Bulbo Seco (°C)", "Temp. Orvalho (°C)", "Temp. Máx (°C)",
    "Temp. Mín (°C)", "Orvalho Máx (°C)", "Orvalho Mín (°C)",
    "Umidade Rel. Máx (%)", "Umidade Rel. Mín (%)",
    "Umidade Rel. Horária (%)", "Vento Direção (°)",
    "Rajada Máx (m/s)", "Velocidade Vento (m/s)"
]

In [10]:
print(df_clima.head())

  Precipitação (mm) Pressão (mB) Pressão Máx (mB) Pressão Mín (mB)  \
0                 0        937,5            937,5            936,8   
1                 0        938,4            938,4            937,5   
2                 0        938,9            938,9            938,4   
3                 0        938,8              939            938,8   
4                 0        938,3            938,8            938,3   

  Radiação (Kj/m²) Temp. Bulbo Seco (°C) Temp. Orvalho (°C) Temp. Máx (°C)  \
0              NaN                  24,8               19,9           25,6   
1              NaN                  23,7               20,2           24,8   
2              NaN                  23,4               20,3           23,9   
3              NaN                  23,8               19,9           24,7   
4              NaN                  23,3               20,2           24,2   

  Temp. Mín (°C) Orvalho Máx (°C) Orvalho Mín (°C)  Umidade Rel. Máx (%)  \
0           24,5               20 

Criar coluna única de Data e Hora para a tabela de dados climáticos e remove as anteriores.

In [7]:
df_clima['Data Hora'] = pd.to_datetime(df_clima['Data'] + ' ' + df_clima['Hora UTC'].str.strip(' UTC'), format='%Y/%m/%d %H%M')
df_clima.drop(columns=['Data', 'Hora UTC'], inplace=True)


Cria uma nova coluna de Data usando apenas o mês e o ano de cada linha da tabela

In [8]:
df_clima['Data'] = df_clima['Data Hora'].dt.strftime('%Y/%m')

In [16]:
def format_value(value):
  try:
    return pd.to_numeric(value.str.replace(',', '.'), errors='coerce')
  except:
    return value

In [9]:
clima_agrupado = df_clima.groupby('Data').agg({
    "Precipitação (mm)": 'sum',           # Soma da precipitação mensal
    "Pressão (mB)": 'mean',              # Média da pressão
    "Pressão Máx (mB)": 'max',           # Máxima da pressão
    "Pressão Mín (mB)": 'min',           # Mínima da pressão
    "Radiação (Kj/m²)": 'sum',           # Soma da radiação
    "Temp. Bulbo Seco (°C)": 'mean',     # Temperatura média
    "Temp. Orvalho (°C)": 'mean',        # Orvalho médio
    "Temp. Máx (°C)": 'max',             # Máxima temperatura
    "Temp. Mín (°C)": 'min',             # Mínima temperatura
    "Orvalho Máx (°C)": 'max',           # Máxima do orvalho
    "Orvalho Mín (°C)": 'min',           # Mínima do orvalho
    "Umidade Rel. Máx (%)": 'max',       # Máxima da umidade relativa
    "Umidade Rel. Mín (%)": 'min',       # Mínima da umidade relativa
    "Umidade Rel. Horária (%)": 'mean',  # Média da umidade relativa
    "Vento Direção (°)": 'mean',         # Direção média do vento
    "Rajada Máx (m/s)": 'max',           # Máxima rajada de vento
    "Velocidade Vento (m/s)": 'mean'     # Velocidade média do vento
}, numeric_only=True).reset_index()


TypeError: agg function failed [how->mean,dtype->object]

In [20]:
print(clima_agrupado)

       Data  Precipitação (mm)  Pressão (mB)  Pressão Máx (mB)  \
0   2023/10              132.4    936.541129             941.7   
1   2023/11              120.8    936.545694             942.0   
2   2023/12              182.4    936.742625             941.4   
3   2024/01              228.0    937.014073             942.3   
4   2024/02              286.8    938.347414             943.7   
5   2024/03              287.0    938.023925             942.0   
6   2024/04              323.4    938.396389             941.8   
7   2024/05                1.2    940.378091             946.8   
8   2024/06                0.0    941.202500             947.4   
9   2024/07                0.0    942.712903             948.3   
10  2024/08                0.0    941.693817             948.3   
11  2024/09                3.0    938.363472             943.9   
12  2024/10              233.4    937.177688             944.0   

    Pressão Mín (mB)  Radiação (Kj/m²)  Temp. Bulbo Seco (°C)  \
0         